In [1]:
from databases import BDS,Crm,BDSMem,PosicaoDm1, Boletador, CVM
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from funcoes_datas import FuncoesDatas
import pandas as pd
from emailer import Email, EmailLer
import numpy as np

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [3]:
##Puxa as classes do databases (consultas nas bases sql da empresa)
crm = Crm()
crm_fundos = Crm(load_cadfundos=True)
bds = BDS()
fdt = FuncoesDatas()
bol = Boletador()
posicao = PosicaoDm1()
cvm = CVM()

#retorna posição de todas as carteiras + fundos
posicao = PosicaoDm1()

#Retorna a data de hoje
hoje = fdt.hoje()

#Retorna as informações dos fundos
fundos = crm_fundos.fundo_cadastro()

#Data_ini_boletas
data_ini = pd.datetime(2024,1,1)

In [33]:
#Função que analisa o emissor
def entra_comp_nome(ativos):
    ativos_isento = ['fidc','fia','fiagro','fia','ishares']
    if any(ativo in ativos for ativo in ativos_isento):
        isento = True
    else:
        isento = False
    return isento    

In [36]:
def entra_comp(Classe,NomeProduto,TipoProduto):
    produto = NomeProduto.lower()
    produto = NomeProduto.split()
    if (Classe == 'R Variável') or (Classe == 'P. Equity') or (Classe=='Crédito Alternativo') or (Classe=='Infraestrutura Ilíquidos'):
        comp='sim'
    elif ('FIP' in produto) or ('FIDC' in produto) or ('FIA' in produto):
        comp ='sim'
    elif (TipoProduto=='FIDC') or (TipoProduto=='FII') or (TipoProduto=='FIP'):
        comp ='sim'        
    elif entra_comp_nome(produto)== True:
         comp ='sim'        
    else:
        comp ='não'
        
    return comp    
    

In [42]:
ativos_desconsiderar = ['COMP','CAIXA','Ajustes']
po_cadastro = cadastro= posicao.po_cadastro()[['NomeContaCRM','TipoCobranca','TipoCondominio','PerfilInvNome',]]

po_ativos = posicao.ativos_cadastro_po_ativos().reset_index()[['GuidAtivo','TipoProduto']]

po_posicao = posicao.posicao_fundos_all(hoje,False)
po_posicao = po_posicao.loc[~po_posicao['TipoProduto'].isin(ativos_desconsiderar)]
po_posicao['GuidContaCRM'] = po_posicao['GuidContaCRM'].str.lower()
po_posicao = po_posicao[['NomeContaCRM','GuidProduto','NomeProduto','Classe','PesoFuturo']]

po_posicao = pd.merge(left=po_posicao,right=po_cadastro,on='NomeContaCRM',how='left')
po_posicao = pd.merge(left=po_posicao,right=po_ativos,left_on = 'GuidProduto',right_on='GuidAtivo',how='left').rename(columns={'NomeProduto_y':'NomeProduto','Classe_y':'Classe'})
po_posicao['Entra_comp'] = po_posicao.apply(lambda row : entra_comp(row['Classe'],row['NomeProduto'],row['TipoProduto']),axis=1)

#Avaliar Fundos
po_posicao = po_posicao[po_posicao['Entra_comp']=='sim']
po_posicao = po_posicao.groupby(['NomeContaCRM','TipoCobranca','TipoCondominio']).sum().reset_index()[['NomeContaCRM','TipoCobranca','TipoCondominio','PesoFuturo']]
fundos_s_come = po_posicao[po_posicao['PesoFuturo']>0.95].sort_values(by=['NomeContaCRM'])
fundos_s_come_FIM = fundos_s_come[fundos_s_come['NomeContaCRM'].str.contains('FIM')]

#Retorna os fundos SuperFim
fundos_s_come.to_excel('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/SUPERFIM/FundosSuperFim.xlsx')


In [46]:
fundos_s_come

,NomeContaCRM,TipoCobranca,TipoCondominio,PesoFuturo
22,_Algarve FIM CrPr IE,FundoExclusivo,ABERTO,0.972588
80,_CFO Spectra IV Brasil FIP Multiestrategia,VeiculoPublico,FECHADO,0.962332
87,_CSHG Seta 2 FIM CP IE (Monitorado),Outro,ABERTO,0.965928
100,_Carro FIA BDR Nivel I,FundoExclusivo,FECHADO,0.998096
127,_Cougar FIC FIA,Outro,ABERTO,0.987395
132,_Curie FIM CrPr IE,FundoExclusivo,ABERTO,1.261457
155,_Eupar II FIA IE,FundoExclusivo,ABERTO,0.966163
202,_Gold Premium FIC FIM CrPr,Outro,ABERTO,1.796814
221,_Ibovespa Alocação FIA,VeiculoPublico,ABERTO,0.989723
238,_JBFO Brazil Master FIA BDR Nivel I,Outro,ABERTO,0.975181
